### Some Twitter Data!
Daniel Zheng

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt
import tweepy as tp
import pandas as pd

In [8]:
key, secret = open('api_key.txt').read().splitlines()

In [9]:
auth = tp.OAuthHandler(consumer_key=key, consumer_secret=secret)
api = tp.API(auth)

In [13]:
trump = api.user_timeline(id='realDonaldTrump') # last 20 tweets

In [24]:
# let's take a look at the first one
tweet = trump[0]

Help on Status in module tweepy.models object:

class Status(Model)
 |  Method resolution order:
 |      Status
 |      Model
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __ne__(self, other)
 |      Return self!=value.
 |  
 |  destroy(self)
 |  
 |  favorite(self)
 |  
 |  retweet(self)
 |  
 |  retweets(self)
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  parse(api, json) from builtins.type
 |      Parse a JSON object into a model instance.
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __hash__ = None
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Model:
 |  
 |  __getstate__(self)
 |  
 |  __init__(self, api=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |